<h1>NTRU public key cryptosystem</h1>
In this notebook, we are going to explore the NTRU cryptosystem from the theory to the application, so here's the summary:<br>
<ul>
    <li>Introduction</li>
    <li>Theoritical Aspects</li>
    <li>A Sagemath-based Implementation</li>
    <li>The lattice Cryptanalysis Against NTRU </li>
    <li>Ressources</li>
</ul>

<h2>Introduction</h2>
NTRU is  ring-based a public key cryptosystem, it was introduced by Jeffrey Hoffstein, Jill Pipher, Joseph H. Silverman(<a href="https://www.ntru.org/f/hps98.pdf"> original paper</a>), NTRU has many version, in this notebook, we will focus on the polynomial version, in a basic term,the plaintext and ciphertext will  both encoded as polynomials. The main textbook support for this notebook is the great book "An Introduction to Mathematical Cryptography" written by the inventor of NTRU.

<h2>Theoritical Aspects</h2>
This version of NTRU is based on polynomial rings, 
Where :
<img src="Images/Poly_ring.png"  style="width:400px ;height:50px"> 

<ul>
<li>$R$: is a quotient ring of the ring of polynomials with integer coefficients where every polynomial which is divisible by  the polynomial $X^{n}-1$ will be the zero element , the same thing on the ring $\mathbb{Z}/n\mathbb{Z}$ where the multiple of n are all considered as zero.  </li>
    <li>$R_{q}$ and $R_{p}$ are the same as $R$, but the coefficients of the polynomials are taken modulo q (Resp p)</li>
</ul>
and t(d,d) : refers to the ternanry polynomial notation.
<img src="Images/Tdd.png"  style="width:400px ;height:100px"> 

the NTRU cryptosystem is illustred on the figure below :
<img src="Images/Ntru_gen.png"  style="width:400px ;height:400px"> 

a small remark: the plaintext (resp ciphertext), should be a polynomial with coefficient on the interval $[ \frac{-p}{2},\frac{p}{2}]$
, this operatin is called "Centerlifting".



<h2>A Sagemath-based Implementation</h2>
Here's a basic implementation of the NTRU cryptosystem using sagemath and python, and for the examples, i have used a binary encoding scheme, so, the binary representation of the plaintext will be the coefficient of its binary polynomial representation.

In [1]:
import random



In [2]:
class NTRU:
    f_x = None
    g_x = None
    Fp_x = None
    Fq_x = None
    hx = None
    R = None
    Rq = None
    Rp = None
    def __init__(self,N,p,q,d):
        self.N = N
        self.p = p
        self.q = q
        self.d = d
        
    def random_poly(self,N,d1,d2):
        coef_list = [1]*d1+[-1]*d2+[0]*(N-d1-d2)
        random.shuffle(coef_list)
        random.shuffle(coef_list)
        
        return coef_list
        

    def keygen(self):
        RR.<x>= ZZ[]
        RRq.<x>= GF(self.q)[]
        RRp.<x>=  GF(self.p)[]
        R = RR.quotient(x ^ self.N - 1) 
        Rq = RR.change_ring(Integers(self.q)).quotient(x^self.N - 1)
        Rp = RR.change_ring(Integers(self.p)).quotient(x^self.N - 1)
        while True:
            try:

                f_x = R(self.random_poly(self.N, self.d + 1, self.d))
                g_x = R(self.random_poly(self.N, self.d, self.d))
                Fp_x = Rp(lift(1/Rp(f_x)))
                Fq_x = Rq(lift(1/Rq(f_x)))
                break
            except: 
                continue

        assert Fp_x * f_x == 1 and Fq_x * f_x == 1
        h_x = Rq( Fq_x * g_x)
        self.f_x,self.g_x,self.Fp_x,self.Fq_x,self.h_x = f_x,g_x,Fp_x,Fq_x,h_x
        self.R,self.Rq,self.Rp = R,Rq,Rp
        

    def encrypt(self,m:list):
        self.keygen()  
        r_x= self.Rq(self.random_poly(self.N,self.d,self.d))
        m_x= self.Rp(m)
        m_x = m_x.lift()
        m_x = self.Rq(m_x)
        e_x =  self.Rq(self.p *self.h_x*r_x + m_x)
        return e_x.list(),self.h_x.list()
    

    def decrypt(self,e:list):
        e_x = self.Rq(e)
        a_x = self.Rq(self.f_x * e_x)
        a_x = ZZ['x']([coeff.lift_centered() for coeff in a_x.lift()]) #Centerlift
        b_x = self.Rp(self.Fp_x * self.R(a_x))
        return b_x.list()





In [3]:
def str2bin(s):
    return ''.join(bin(ord(i))[2:].zfill(8) for i in s)

def bin2str(a):
    return ''.join(chr(int(a[i:i+8],2)) for i in range(0,len(a),8))
m =  'this is a simple 32bytes message'
assert len(m)*8 == 256

In [4]:
N,p,q,d = 2**8,2,29,2

assert gcd(N,q)==1 and gcd(p,q)==1 and q>(6*d+1)*p
cipher = NTRU(N,p,q,d)
m = list(str2bin(m))
e ,h =  cipher.encrypt(m)



In [5]:
dec = ''.join(str(x) for x in cipher.decrypt(e))
print(bin2str(dec))


this is a simple 32bytes message


<h2>The lattice Cryptanalysis Against NTRU</h2>
Like every cryptosystem, NTRU is also vulnerable, and it can cryptanalysed using lattice reduction techniques. A lattice is  a vector space defined over  a ring, lattices are out of the scope of this notebook, so we are not going to go in depth on the cryptanalysis details. The technique is based on the short vector problem in lattices <a href="https://en.wikipedia.org/wiki/Lattice_problem">SVP</a>, and if the parameters (N,p,q,d) are chosen in a bad way, the private key vector (f,g) will be the short vector on the lattice associated to the public key H.

<img src="Images/Lattice.png" alt="The lattice associated to h" style="width:500px ;height:500px"   >
The relation between the associated lattice and the private key is illustred below
<img src="Images/f_g.png"  style="width:400px ;height:100px" >

<img src="Images/prop_1.png"  style="width:400px ;height:100px" >
<img src="Images/prop2.png"  style="width:400px ;height:200px" >

In [6]:
N,p,q,d = 8,2,29,2

assert gcd(N,q)==1 and gcd(p,q)==1 and q>(6*d+1)*p
weak_cipher = NTRU(N,p,q,d)
m ='{'
m = list(str2bin(m))
e ,h=  weak_cipher.encrypt(m)

In [7]:
#the attack
def LLL_attack(h):
    l = len(h)
    A, I = matrix.circulant(h), matrix.identity(l)
    H = block_matrix(ZZ, [[I, A], [0 * I, q * I]])
    H = matrix(ZZ, H)
    print(f"the associated lattice:")
    print(H.str())
    H_red = H.LLL()
    return H_red
H_red = LLL_attack(h)
H_red

the associated lattice:
[ 1  0  0  0  0  0  0  0| 0 13 20  9 24 21 20 17]
[ 0  1  0  0  0  0  0  0|17  0 13 20  9 24 21 20]
[ 0  0  1  0  0  0  0  0|20 17  0 13 20  9 24 21]
[ 0  0  0  1  0  0  0  0|21 20 17  0 13 20  9 24]
[ 0  0  0  0  1  0  0  0|24 21 20 17  0 13 20  9]
[ 0  0  0  0  0  1  0  0| 9 24 21 20 17  0 13 20]
[ 0  0  0  0  0  0  1  0|20  9 24 21 20 17  0 13]
[ 0  0  0  0  0  0  0  1|13 20  9 24 21 20 17  0]
[-----------------------+-----------------------]
[ 0  0  0  0  0  0  0  0|29  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0| 0 29  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0| 0  0 29  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0| 0  0  0 29  0  0  0  0]
[ 0  0  0  0  0  0  0  0| 0  0  0  0 29  0  0  0]
[ 0  0  0  0  0  0  0  0| 0  0  0  0  0 29  0  0]
[ 0  0  0  0  0  0  0  0| 0  0  0  0  0  0 29  0]
[ 0  0  0  0  0  0  0  0| 0  0  0  0  0  0  0 29]


[ 3 -1  0 -1 -3  1  0  1 -1  0  0  0  1  0  0  0]
[ 1  3 -1  0 -1 -3  1  0  0 -1  0  0  0  1  0  0]
[ 0  1  3 -1  0 -1 -3  1  0  0 -1  0  0  0  1  0]
[ 1  0  1  3 -1  0 -1 -3  0  0  0 -1  0  0  0  1]
[ 0  0 -1 -3  2  3 -1  0  1 -1 -1  2  1 -2 -1  1]
[-2  1  0  0  0  0 -1 -4  3  1  0  1  2  1  0  2]
[ 0 -1 -3  1 -1 -3  1  0  0  1  3  1  0  2  2  1]
[-1  0 -4  2  0  4 -2  1  0 -1  1  1  0 -2  0  1]
[ 1  0  0  0  0  0 -1 -3 -1  2 -2  3 -1  2 -2  4]
[-2 -2  3  1 -3 -3 -1 -1  2 -3 -2 -1  2 -3 -1  0]
[ 0  0  0  1  2  3  1  0  0 -2 -1 -1 -3  5  1 -1]
[-3 -1  3  3  1  1  2  2  2  1 -2  3  1  0 -2  3]
[-2  3  2  0 -4 -1 -1 -2 -2 -4  1 -2 -2 -3  2 -1]
[ 0  0  1  2  3  1  0  0 -2 -1 -1 -3  5  1 -1  0]
[ 0  0  0  0 -1 -2 -3 -1  1  0  2  1  1  3 -5 -1]
[-1  0  0  0  0 -1 -2 -3 -1  1  0  2  1  1  3 -5]

In [8]:
RR.<x> = ZZ[]
RRq.<x> = GF(q)[]
RRp.<x> =  GF(p)[]
R = RR.quotient(x ^ N - 1) 
Rq = RR.change_ring(Integers(q)).quotient(x^N-1)
Rp = RR.change_ring(Integers(p)).quotient(x^N-1)

    


In [9]:
from collections import deque

def rotate(f_x:list,n):
     f_x = deque(f_x)
     f_x.rotate(-n)
     return list(f_x)

def getMode(l: list):
    return max(set(l), key=l.count)

def decrypt(e:list,f_x:list):
        f_x = R(f_x)
        Fp_x = Rp(lift(1/Rp(f_x)))
        e_x = Rq(e)
        a_x = Rq(f_x * e_x)
        a_x = ZZ['x']([coeff.lift_centered() for coeff in a_x.lift()])
        b_x = Rp(Fp_x * R(a_x))
        b_x =''.join(str(x) for x in b_x)
        return bin2str(b_x)


<h3>Remark</h3>
Since the private key space is relatively small, there's is $\frac{N!}{(d+1)!(d!)(N-2d-1)!} = \frac{8!}{3!2!3!} = 560$ ways to choose f, and this is obvioulsy bruteforcable !. And due to some noise on the random function, the short vector won't be necessarily on the top, so i have tried to take the mode of decrypted result to find the desired plaintext.

In [10]:
def getResult():
    out = []
    for i in range(2 * N):
        for j in range(N):
            f_x = rotate(list(H_red[i])[:N], j)
            try:
                byte = decrypt(e, f_x)
                out.append(byte)
            except:
                continue
    
    return  getMode(out)



print(getResult())


{


<h2> Ressources</h2>
Here's some good ressources to learn more about the NTRU cryptosystem:
<ul>
    <li><a href="https://www.youtube.com/@tanjalangepost-quantumcryp2802">Tanja lanje youtube channel</a> </li>
    <li><a href="https://asecuritysite.com/"> Prof Bill Buchanan</a></li>
    <li><a href="https://www.ntru.org/f/hps98.pdf">Original Paper</a> </li>
    <li><a href="https://link.springer.com/book/10.1007/978-0-387-77993-5">An Introduction to Mathematical Cryptography</a> </li>
    <li><a href="https://www.sagemath.org/">Sagemath </a> </li>
    
</ul>